In [1]:
import pandas as pd
import os
import missingno as msno
%matplotlib inline
#%matplotlib notebook
from IPython.display import display

In [2]:
%%time
wdi_df = pd.read_excel(os.path.join("..","data","WDIEXCEL.xlsx"))
series_df = pd.read_excel(os.path.join("..","data","WDIEXCEL.xlsx"),sheet_name='Series')

CPU times: user 1min 1s, sys: 5.22 s, total: 1min 7s
Wall time: 1min 25s


In [19]:
wdi_merge=pd.merge(wdi_df,series_df,how='left',left_on='Indicator Code',right_on='Series Code')
wdi_ghg=wdi_merge.drop(['Indicator Name_y', 'Short definition', 'Long definition',
       'Unit of measure', 'Periodicity', 'Base Period', 'Other notes',
       'Aggregation method', 'Limitations and exceptions',
       'Notes from original source', 'General comments', 'Source',
       'Statistical concept and methodology', 'Development relevance',
       'Related source links', 'Other web links', 'Related indicators',
       'License Type','Series Code'],axis=1)
wdi_ghg=wdi_ghg[wdi_ghg['Indicator Name_x'].isin([
'Agricultural methane emissions (thousand metric tons of CO2 equivalent)',
'Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent)',
'Energy use (kg of oil equivalent per capita)',
'GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent)',
'GDP per unit of energy use (PPP $ per kg of oil equivalent)',
'GHG net emissions/removals by LUCF (Mt of CO2 equivalent)',
'HFC gas emissions (thousand metric tons of CO2 equivalent)',
'Methane emissions (kt of CO2 equivalent)',
'Methane emissions in energy sector (thousand metric tons of CO2 equivalent)',
'Nitrous oxide emissions (thousand metric tons of CO2 equivalent)',
'Nitrous oxide emissions in energy sector (thousand metric tons of CO2 equivalent)',
'Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)',
'PFC gas emissions (thousand metric tons of CO2 equivalent)',
'SF6 gas emissions (thousand metric tons of CO2 equivalent)',
'Total greenhouse gas emissions (kt of CO2 equivalent)'])]
wdi_ghg.to_csv(os.path.join("..","data",'wdi_ghgs.csv'))
wdi_ghg

,Country Name,Country Code,Indicator Name_x,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Topic
63,Africa Eastern and Southern,AFE,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,333710.000000,332070.000000,332310.000000,337680.000000,343020.00000,341240.0,343700.0,NaN,NaN,Environment: Emissions
65,Africa Eastern and Southern,AFE,Agricultural nitrous oxide emissions (thousand...,EN.ATM.NOXE.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,242050.000000,239560.000000,239500.000000,241570.000000,245360.00000,246470.0,246400.0,NaN,NaN,Environment: Emissions
382,Africa Eastern and Southern,AFE,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,NaN,NaN,NaN,NaN,NaN,NaN,...,728.721222,725.668196,740.766343,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Energy production & use
477,Africa Eastern and Southern,AFE,GDP per unit of energy use (constant 2017 PPP ...,EG.GDP.PUSE.KO.PP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.551395,5.656386,5.669844,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Energy production & use
478,Africa Eastern and Southern,AFE,GDP per unit of energy use (PPP $ per kg of oi...,EG.GDP.PUSE.KO.PP,NaN,NaN,NaN,NaN,NaN,NaN,...,4.995249,5.223737,5.406703,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Energy production & use
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383312,Zimbabwe,ZWE,Nitrous oxide emissions in energy sector (thou...,EN.ATM.NOXE.EG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,1150.000000,1270.000000,1390.000000,1510.000000,1540.00000,1560.0,1590.0,NaN,NaN,Environment: Emissions
383335,Zimbabwe,ZWE,"Other greenhouse gas emissions, HFC, PFC and S...",EN.ATM.GHGO.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,1827.732117,1972.408020,1933.483643,2499.469604,1587.37207,NaN,NaN,NaN,NaN,Environment: Emissions
383381,Zimbabwe,ZWE,PFC gas emissions (thousand metric tons of CO2...,EN.ATM.PFCG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Emissions
383643,Zimbabwe,ZWE,SF6 gas emissions (thousand metric tons of CO2...,EN.ATM.SF6G.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Emissions


In [9]:
#1) Num countries that have the data
## of years that have the data
wdi_final

,Country Name,Country Code,Indicator Name_x,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Topic


In [18]:
wdi_merge=pd.merge(wdi_df,series_df,how='left',left_on='Indicator Code',right_on='Series Code')
wdi_ghgs=wdi_merge.drop(['Indicator Name_y', 'Short definition', 'Long definition',
       'Unit of measure', 'Periodicity', 'Base Period', 'Other notes',
       'Aggregation method', 'Limitations and exceptions',
       'Notes from original source', 'General comments', 'Source',
       'Statistical concept and methodology', 'Development relevance',
       'Related source links', 'Other web links', 'Related indicators',
       'License Type','Series Code'],axis=1)
wdi_ghgs=wdi_ghgs[wdi_ghgs['Indicator Name_x'].isin(['Agricultural methane emissions (thousand metric tons of CO2 equivalent)'])]
wdi_ghgs

,Country Name,Country Code,Indicator Name_x,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Topic
63,Africa Eastern and Southern,AFE,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,333710.0,332070.0,332310.0,337680.0,343020.0,341240.0,343700.0,NaN,NaN,Environment: Emissions
1506,Africa Western and Central,AFW,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,175140.0,181460.0,184600.0,191240.0,203680.0,204690.0,208280.0,NaN,NaN,Environment: Emissions
2949,Arab World,ARB,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,121460.0,121830.0,122590.0,121580.0,122500.0,121660.0,122480.0,NaN,NaN,Environment: Emissions
4392,Caribbean small states,CSS,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,2410.0,2620.0,2720.0,2770.0,2440.0,2570.0,2480.0,NaN,NaN,Environment: Emissions
5835,Central Europe and the Baltics,CEB,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,36840.0,36740.0,37030.0,37280.0,36980.0,37150.0,37210.0,NaN,NaN,Environment: Emissions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376686,Virgin Islands (U.S.),VIR,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Emissions
378129,West Bank and Gaza,PSE,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environment: Emissions
379572,"Yemen, Rep.",YEM,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,4680.0,4750.0,4830.0,4770.0,4630.0,4600.0,4880.0,NaN,NaN,Environment: Emissions
381015,Zambia,ZMB,Agricultural methane emissions (thousand metri...,EN.ATM.METH.AG.KT.CE,NaN,NaN,NaN,NaN,NaN,NaN,...,11320.0,11470.0,11210.0,11310.0,11000.0,10450.0,10710.0,NaN,NaN,Environment: Emissions
